In [19]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import matplotlib.axis as ax
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import unicodedata
from scipy.optimize import curve_fit
import pycountry as pc
import math
import lxml
from jupyterthemes import jtplot
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from statistics import mean
import operator
from PyPDF2 import PdfFileReader

DATA_DIR = "./data/"
DEMO_DIR = "./population/"
DEM = "https://www.populationpyramid.net/"
WEB = "https://www.worldometers.info/coronavirus/"
WHO_DIR = "./who_cm/"
AGE = 0
SEX = 1
COM = 2
MIG = pd.read_csv("MIG.csv",low_memory=False)
idx = MIG[MIG["Country of birth/nationality"] =='Former Czechoslovakia'].index
MIG.drop(idx,inplace=True)
jtplot.style(theme='grade3')
jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')
jtplot.style(ticks=True, grid=False, figsize=(8, 6.5))
np.warnings.filterwarnings('ignore')
DEM_DICT = {}
IMM_DICT = {}

# COVID-19 expansion tracking and Mathematical Model.

### 0. Introduction and motivation

SARS-CoV-2 is a virus belonging to the coronaviruses family that was first detected in Hubei province, Mainland China, in December 2019 and triggered an ongoing global pandemia.
Because of the high $R_0$ (estimated between 2 and 3) and the asymptomatic nature of the incubation period, it's necessary to develop mathematical models for prediction, that will prevent further consecuencies by anticipating expansion rate.

### 1. Preliminary analysis

This section is intended to act like a toolbox providing access to resources and data management functions, visualization tools will also be provided.

#### 1.1 Time Tracking and real time evolution

In [20]:
columns = ["Country/Region","Confirmed","Deaths","Recovered"]

In [21]:
def timeEvolution(country):
    data_files = sorted(os.listdir(DATA_DIR))
    """
    Returns a pandas DataFrame with information of a country (Confirmed,Deaths,Recovered) by day
    since the beginig of the outbreak.
    
    params:
    
        -country (String): target country
    
    returns:
        -Pandas DataFrame if information is available , else None.
    """
    dic = []
    for file in data_files:
        key = file.split(".csv")[0]
        df = pd.read_csv(DATA_DIR + file)
        indexes = df["Country/Region"] == country #rows associated with country
        if any(indexes):
            country_data = df[df["Country/Region"] == country][columns].sum() #all records of that country
            dic.append([key]+list(country_data[columns[1:]]))
        else: #no rows
            dic.append([key,0,0,0])
    
    df = pd.DataFrame(dic)
    df.columns = ["Date"]+columns[1:]
    return df
        
def worldTimeEvolution(): #about 20s
    data_files = sorted(os.listdir(DATA_DIR))
    """
    Returns a dictionary indexed by country of DataFrames with information about countries since outbreak
    
    returns:
        -dictionatry of pandas DataFrames
    """
    dic = {}
    countries = pd.read_csv(DATA_DIR+data_files[-1])["Country/Region"].unique()
    for country in countries:
        dic[country] = timeEvolution(country)  
    return dic

In [ ]:
def graphProgression(countryDF,param): #TODO: x axis problem with dates, too messy
    """
    Graphs the progression in time of a given parameter regarding country info
    
    params:
        -countryDF (pandas.DataFrame): info about the country (see timeEvolution or worldTimeEvolution)
        -param (String): "Confirmed" -> confirmed cases of COVID-19 in country
                         "Deaths" -> deaths by COVID-19 in country
                         "Recovered" -> recovered patients in country 

    """
    x = range(len(countryDF["Date"]))
    y = list(countryDF[param].astype(int))
    plt.plot(x,y)

    y_max = max(y)
    if y_max < 10:
        plt.yticks(range(0,max(y)+1,1))
    else:
        plt.yticks(range(0,max(y)+1,max(y)//10))
    plt.title("{} COVID-19".format(param))
    plt.ylabel(param)
    plt.xlabel("Days from outbreak")
    plt.show()


graphProgression(timeEvolution("Spain"),"Confirmed")

In [25]:
def realTimeEvolution():
    """
    Returns a Pandas DataFrame with infomration about the real time expansion of the virus.
    
    returns:
        -pandas Dataframe
    """
    cols = ["Country","Total Cases","New Cases","Total Deaths","New Deaths","Active Cases","Total Recovered","Critical","Tot Cases/1 M Pop"]
    page = requests.get(WEB)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('table')[0]
    df =  pd.read_html(str(table),header=0)[0]
    df.columns = cols
    df.drop(df.tail(1).index, inplace=True)
    df.fillna(0,inplace=True)
    return df

def updateData():
    """
    Stores today's evolution data in csv format
    
    """
    df = realTimeEvolution()[["Country","Total Cases","Total Deaths","Total Recovered"]]
    df.columns=columns
    now = datetime.now()
    df.to_csv(DATA_DIR+now.strftime("%Y-%m-%d-%H-%M.csv"))
#updateData()

##### 1.1.0. Dictionaries for resource mining

In [26]:
# Functions to initialize dictionaries for resource mining
def lcs(s1,s2): 
    """
    Returns the longest common substring between two strings.
    
    params:
        - s1 (String): first string
        - s2 (String): second string
    returns:
        - Longest Common Substring (LCS)
    """
    m = [[0] * (1 + len(s2)) for i in range(1 + len(s1))]
    longest, x_longest = 0, 0
    for x in range(1, 1 + len(s1)):
        for y in range(1, 1 + len(s2)):
            if s1[x - 1] == s2[y - 1]:
                m[x][y] = m[x - 1][y - 1] + 1
            if m[x][y] > longest:
                longest = m[x][y]
                x_longest = x
            else:
                m[x][y] = 0
    return s1[x_longest - longest: x_longest]

def countryPred(country,possibleCountries):
    """
    Returns a prediction of a country name based on lcs similarity.
    
    params:
        - country (String): country name
        - possibleCountries (List): list with different ways to call the same country
    returns:
        - Prediction string
    """
    pred = sorted(possibleCountries,key=lambda x: len(lcs(country.replace(" ","").replace("China,","").replace("North","").replace(" China,",""),x.replace(" ","").replace("China,","").replace(" China,","").replace("North",""))),reverse=True)[0]
    return pred


def genDic(option):
    """
    Returns a dictionary of different ways to call a country specifying an option.
    
    params:
        - option (String): IMM for Immigration data countries or DEM for Demography data countries
    returns:
        - Dictionary of countries
    """
    realtime_countries = list(realTimeEvolution()['Country'].values)
    if option == "IMM":
        possibleCountries = MIG["Country of birth/nationality"].unique()
    elif option == "DEM":
        possibleCountries = [file.split(".csv")[0] for file in os.listdir(DEMO_DIR)]
    
    countries = [(country,countryPred(country,possibleCountries)) for country in realtime_countries]
    trash = ['Monaco', 'San Marino', 'Faeroe Islands', 'Andorra', 'Diamond Princess', 'Taiwan', 'Gibraltar', 'Vatican City', 'Liechtenstein']
    dic = {}
    for country,pred in countries:
        if country not in pred:
            if country == "USA":
                if option == "IMM":
                    pred = "United States" #change
                if option == "DEM":
                    pred = "United States of America"
            elif country == "UK":
                pred = "United Kingdom"
            elif country == "UAE":
                pred = "United Arab Emirates"
            elif country in trash:
                continue
            else:
                dic[country] = pred

        dic[country] = pred

    return dic

#NOTE: Take her to Alice in Wonderland Ballet

IMM_DICT = genDic("IMM")
DEM_DICT = genDic("DEM")

#### 1.2. Demography by country

In [27]:

def saveDemography():
    """
    Mines https://www.populationpyramid.net and saves population pyramids 
    in csv files stored in the population directory
    """
    #TODO mine https://www.populationpyramid.net/<country>/2019/ , they have download by link, need to know country number, explore HTML + wget
    DEM = "https://www.populationpyramid.net/"
    html_page = requests.get(DEM)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    page_data = soup.find_all(class_='countryLink')
    l = []
    if not os.path.exists(DEMO_DIR):
        os.makedirs(DEMO_DIR)
    for country in page_data:
        d = {}
        aux = country.contents[0]
        if (aux == "Australia/New Zealand"):
            aux = "New Zealand"
        d['Country'] = aux
        d['ID'] = country['country']
        l.append(d)
        
    for country in l:
        c, ids = country.get("Country"), country.get("ID")
        CSV = "https://www.populationpyramid.net/api/pp/"+ids+"/2019?csv=true"
        r = requests.get(CSV)
        filename = c+".csv"
        file_path = os.path.join(DEMO_DIR, filename)
        if r.ok:
            print(filename + " : saving data to ... ", os.path.abspath(DEMO_DIR))
            with open(file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024 * 8):
                    if chunk:
                        f.write(chunk)
                        f.flush()
                        os.fsync(f.fileno())
        else:  # HTTP status code 4XX/5XX
            print("Download failed: status code {}\n{}".format(r.status_code, r.text))
    

def getDemography(country_info):
    """
    Gets the population pyramid of a given country
    
    params:
        - country_info (string): name of the country (assuming notation)
    returns:
        pandas.DataFrame
    """
    country_info = DEM_DICT[country_info]
    file = country_info+".csv"
    df = pd.read_csv(DEMO_DIR + file)
    return df

def plotDemography(country_info):
    """
    Plots the demography pyramid of a country separeted by sex
    
    params:
        - country_info (string): name of the country (assuming notation)
    """
    df = getDemography(country_info)
    a = df.plot
    a.bar(x='Age',rot=55)

def eqCountry(country):
    """
    Returns the common string to name a country.
    
    params:
        - country (String): second name
    returns:
        - Common way to call the country
    """
    if country == 'Korea, Republic of':
        country = 'S. Korea'
    for k, v in DEM_DICT.items():
        if country in v:
            country = k
    return country
    
def inhabitantsCountry(country):
    """
    Returns the population of a country.
    
    params:
        - country (String): country
    returns:
        - Total sum of inhabitants per country
    """
    data = getDemography(eqCountry(country))
    return (data['M'] + data['F']).sum()
inhabitantsCountry("Italy")
#country_info="S. Korea"
#plotDemography(country_info)


60550092

#### 1.3. Immigration flow information

In [28]:
def __immigration(c_from,c_to):
    """
    Gets information of incoming immigrants from a country to another in time.
    
    params:
        - c_from : country of departure 
        - c_to: country of arrival (assuming notation)
    returns:
        - pandas.DataFrame
    """
    c_to = IMM_DICT[c_to]
    cols = ["Year","Value"]
    df = MIG[(MIG["VAR"]=="B11") & (MIG["Country of birth/nationality"]==c_from) &((MIG["Country"]==c_to))][cols] 
    df.set_index("Year",inplace=True)
    return df

def immigrationTo(country):
    """
    Returns the immigration flow to a country by immigrant nationality
    
    params:
        -country: destination country (assuming notation)
    returns:
        pandas.DataFrame
    """
    l = []
    for c_from in MIG["Country of birth/nationality"].unique():
        if c_from == "Stateless" or c_from == "Total" or c_from == "Unknown":
            continue
        else:
            df = __immigration(c_from,country)
            if not df.empty and df.iloc[-1]["Value"] != 0:
                l.append([c_from,df.iloc[-1]["Value"]])
    ret = pd.DataFrame(l,columns=["Country","People"])
    ret.set_index("Country",inplace=True)
    return ret
            
immigrationTo("USA")

,People
Country,
Afghanistan,19538.0
Albania,5722.0
Algeria,2139.0
Andorra,2.0
Angola,209.0
Antigua and Barbuda,322.0
Argentina,3482.0
Armenia,3437.0
Australia,2906.0


#### 1.4. Hospital Resources

##### 1.4.1. Hospital Beds

In [30]:
def hospitalBeds():
    """
    Returns a Pandas DataFrame with last information about the number of beds per country
    provided by OECD.
    
    returns:
        - Pandas DataFrame with OECD data filtered by last info in descending order
    """
    pd.set_option('display.max_rows', None)
    data = pd.read_csv("HOSPITAL_BEDS.csv")
    data = data.drop(columns=['SUBJECT', 'INDICATOR', 'FREQUENCY'])
    for code in data['LOCATION'].unique():
        data.replace({'LOCATION': code}, pc.countries.get(alpha_3=code).name, inplace=True)
    data = data.sort_values('TIME', ascending=False).groupby('LOCATION', sort=False).first().sort_values('Value', ascending=False).reset_index()
    return data

def bedsCountry(country):
    """
    Returns the specific number of beds per 1000 habitants of a country.
    
    params:
        - country (String): country
    returns:
        - Float number of beds per 1000 habitants
    """
    data = hospitalBeds()
    return float(data[data.LOCATION.eq(country)].Value)


def tableBedsCountry():
    """
    Returns a Pandas DataFrame with population and beds per country.
    
    returns:
        - Panda DataFrame
    """
    data = hospitalBeds()
    countries = data['LOCATION'].tolist()
    countries.remove('Turkey')
    dic = {}
    for c in countries:
        dic[c] = {}
        dic[c]['N. Habitants'] = inhabitantsCountry(c)
        dic[c]['Beds/1000 hab.'] = bedsCountry(c)
        dic[c]['N. Beds'] = bedsCountry(c)/1000*inhabitantsCountry(c)
    df = pd.DataFrame.from_dict(dic, orient='index')
    df.insert(1,"Country",[eqCountry(c) for c in df.index])
    df.set_index("Country",inplace=True)
    return df

#hospitalBeds()
tableBedsCountry()

,N. Habitants,Beds/1000 hab.,N. Beds
Country,,,
Japan,126860299,13.05,1.655527e+06
S. Korea,51225320,12.27,6.285347e+05
Russia,145872257,8.05,1.174272e+06
Germany,83517045,8.00,6.681364e+05
Austria,8955108,7.37,6.599915e+04
Hungary,9684680,7.02,6.798645e+04
Czechia,10689213,6.63,7.086948e+04
Poland,37887770,6.62,2.508170e+05
Lithuania,2759631,6.56,1.810318e+04


##### 1.4.2. SARS-COV-2 Fatality Rates

In [ ]:
def fatalityRate(option):
    """
    Mines https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/
    Note: 3.4% Mortality Rate estimate by the World Health Organization (WHO) as of March 3
    """
    FR = "https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/"
    html_page = requests.get(FR)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    page_age = soup.find_all('table')[0]
    page_sex = soup.find_all('table')[1]
    page_comorbidity = soup.find_all('table')[2]
    if option == 0:
        df = pd.read_html(str(page_age), header=0)[0]
        df.drop('DEATH RATE confirmed cases', axis=1, inplace=True)
    elif option == 1:
        df = pd.read_html(str(page_sex), header=0)[0]
    elif option == 2:
        df = pd.read_html(str(page_comorbidity), header=0)[0]
        df.drop('DEATH RATE confirmed cases', axis=1, inplace=True)
    return df
fatalityRate(COM)
#fatalityRate(AGE)
#fatalityRate(SEX)

#### 1.4.3. General Fatality Rates

In [31]:
def extractFR():
    """
    Mines https://www.who.int/nmh/countries/en/
    """
    FR_WHO = "https://www.who.int/nmh/countries/en/"
    html_page = requests.get(FR_WHO)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    page_data = soup.find_all(target='_new')
    if not os.path.exists(WHO_DIR):
        os.makedirs(WHO_DIR)
    for country in page_data:
        url = "https://www.who.int"+country.get('href').replace(" ", "")
        name = str(country.contents[0]).split(' [pdf 78kb]')[0]
        r = requests.get(url, allow_redirects=True, stream=True)
        print(r.url)
        filename = name+".pdf"
        file_path = os.path.join(WHO_DIR, filename)
        if r.ok:
            print(filename + " : saving data to ... ", os.path.abspath(WHO_DIR))
            with open(file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024 * 8):
                    if chunk:
                        f.write(chunk)
                        f.flush()
                        os.fsync(f.fileno())
        else:  # HTTP status code 4XX/5XX
            print("Download failed: status code {}\n{}".format(r.status_code, r.text))
    

def getFRCountry(country):
    """
    Returns the country Fatality Rate comorbidity
    """
    try:
        filename = country+".pdf"
        file_path = os.path.join(WHO_DIR, filename)
        with open(file_path, 'rb') as f:
            text = PdfFileReader(f).getPage(0).extractText()
        x = text.split('% of the population   ')[1].split('CURRENT TOBACCO SMOKING')[0].split('  ')
        x = [i.replace("\n", "").strip() for i in x]
        x[4] = x[4].replace(x[4][3:], x[4][3:13]+"(Hypertension)")
        x.pop(2)
        x.pop(2)
        x = [i.split('%') for i in x]
        x = [i[::-1] for i in x]
        x = [x[i] for i in [0, 4, 3, 2, 1]]
        df = pd.DataFrame(x)
        df.columns = ['PRE-EXISTING CONDITION', 'DEATH RATE all cases']
        #df.loc[5] = ['no pre-existing conditions',math.nan] o eso o tomar solo 4 primeras
        return df
    except (FileNotFoundError, IOError):
        print("Wrong file or file path")

def peopleAtRisk(country):
    fr_country = list(map(float, getFRCountry(country)['DEATH RATE all cases'].tolist()))
    fr_coronavirus = []
    for i in fatalityRate(COM)['DEATH RATE all cases'].tolist():
        i = float(i.split('%')[0])
        fr_coronavirus.append(i)
    infected = timeEvolution(country)['Confirmed'].tolist()[-1]
    x = list(map(operator.mul, [i/100 for i in fr_country], [j/100 for j in fr_coronavirus[:5]]))
    aux =[i*infected for i in x]
    return int(sum([i*infected for i in x]))
peopleAtRisk('Spain')

422

#### 1.4.4. Available Hospital Beds Estimation

In [ ]:
def hospitalRatio():
    pd.set_option('display.max_rows', None)
    data = pd.read_csv("HFA.csv")
    data = data.drop(columns=['COUNTRY_GRP','SEX'])
    data.dropna(inplace=True)
    for code in data['COUNTRY'].unique():
        data.replace({'COUNTRY': code}, pc.countries.get(alpha_3=code).name, inplace=True)
    data = data.sort_values('YEAR', ascending=False).groupby('COUNTRY', sort=False).first().sort_values('VALUE', ascending=False).reset_index()
    return data

def HRCountry(country):
    data = hospitalRatio()
    data = data.drop(columns='YEAR')
    value = data.loc[data['COUNTRY'] == country]
    percent = float(value['VALUE'])/100
    n_o = (bedsCountry(country)/1000)*inhabitantsCountry(country)
    return percent*n_o

def availableHospitalBeds(country):
    try:
        n_o = (bedsCountry(country)/1000)*inhabitantsCountry(country)
        return int(n_o - (HRCountry(country) + peopleAtRisk(country)))
    except (NameError, TypeError):
        print("Country not found")

availableHospitalBeds("Spain")

### 2 Mathematical models

#### 2.1 Models

In [ ]:


def expModel(x_data,a,b,d,f):
    """
    Logistic model f(x) = a / (b + f^(-x)*d)
    
    params:
        (a,b,d,f) : numbers for modelling
        x_data : numpy array with data
    
    returns:
        numpy array with vallues corresponding to the model predicition
    """
    return d*np.exp(a*x_data + b) + f
    #return a/(b+(f**-(np.array(x_data)**d)))

def logisticModel(x_data,a,b,d,f):
    return a/(b+(f**-(np.array(x_data)**d)))
    
def fit(func,x_data,y_data):
    """
    fits a function to given data
    
    params:
        -func (function): function to adjust
        -x_data (numpy array): x points
        -y_data (numpy array): y points
    """
    popt, pcov = curve_fit(func,x_data,y_data,maxfev = 10000, bounds=([0,0,-np.inf,0], [np.inf]*4))
    return lambda x: func(x,*popt)

def fromFirstCase(country_df,param):
    """
    Returns x and y values for data given a country from the first day when country had 10 or more cases
    
    params:
        -country_df (pandas.DataFrame): evolution of the country (see timeEvolution())
        -param (String): parameter ("Infected,Deaths,Recovered")
    """
    
    df = country_df[country_df[param] > 10]
    if df.empty:
        return None
    else:
        return np.array(range(1,df.shape[0]+1)),np.array(df[param])

def prediction(country,param,days,model,fromNow=False):
    """
    Return the prediction for a given number of day counting from now or the begining of significant cases
    
    params:
        - country (String): country
        - param (String): parameter ("Infected,Deaths,Recovered")
        - days (Int): number of days from now to predict
        - fromNow (Bool): consider from now (Default) or from the begining. 
    returns:
        days data, param data, pandas DataFrame with data desired
    """
    x_data,y_data = fromFirstCase(timeEvolution(country),param)
    adj = fit(model,x_data,y_data)
    if fromNow :
        x = range(len(x_data),len(x_data)+1+days)
    else:
        x = range(1,len(x_data)+1+days)
    
    y = adj(x)
    return x_data,y_data,pd.DataFrame([[day,people] for day,people in zip(x,y)],columns=["Days","People"])




def plotPrediction(x_data,y_data,df):
    """
    Plots the prediciton
    
    params:
        - x_data (numpy array): relevant data (days)
        - y_data (numpy array): relevant data (param)
        - df (pandas DataFrame): dataframe with prediction
    """
    x,y = df["Days"],df["People"]
    plt.plot(x_data,y_data,'go',label="Data")
    plt.plot(x,y,label="Prediction Model")
    plt.title("Prediction")
    plt.xlabel("Days") 
    plt.ylabel("People")
    plt.legend()


def modelR2(param,model):
    l = []
    for country in list(realTimeEvolution()['Country'].values):
        try:
            data = fromFirstCase(timeEvolution(country),param)
            if country == "Diamond Princess" or country == "Japan" or data == None or len(data[0]) < 2:
                continue
            else:
                x_data,y_data = data
                adj = fit(model,x_data,y_data)
                pred = adj(x_data)
                l.append(r2_score(y_data,pred))
        except:
            continue #country does not have more than 100 cases
    return np.mean(l)

def countryR2(country,param,model):
    data = fromFirstCase(timeEvolution(country),param)
    x_data,y_data = data
    adj = fit(model,x_data,y_data)
    pred = adj(x_data)
    return r2_score(y_data,pred)

print(countryR2("Spain","Confirmed",expModel))
x_data,y_data,df = prediction("Spain","Confirmed",10,expModel,fromNow=False)
print(df)
plotPrediction(x_data,y_data,df)


### 2.2. Stats

In [ ]:
def inverse(func,y,x1,x2,tol=0.001):
    assert x1 < x2
    mid = (x2 - x1)/2 
    if mid <= tol or func(x1 + mid) == y :
        return x1+mid
    elif y > func(x1+mid):
        return inverse(func,y,x1+mid,x2,tol)
    else:
        return inverse(func,y,x1,x1+mid,tol)


def t2(country):
    x_data,y_data = fromFirstCase(timeEvolution(country),"Confirmed")
    adj = fit(expModel,x_data,y_data)
    t2_rates = []
    for x in x_data[1:]:
        inv = inverse(adj,2*adj(x),x_data[0],x_data[-1]+100)
        t2_rates.append((x,inv-x))
    return pd.DataFrame(t2_rates,columns=["Day (>100 cases)","T2"])
    
t2("Spain")
    

In [ ]:
####WORKING
def ageAffected(country_info):
    demography = getDemography(country_info)
    ageFatality = fatalityRate(AGE)
    data = []
    data.extend([[x]*2 if x != "80+ years old" else [x]*5 for x in ageFatality["AGE"]])
    data = sorted([item for sublist in data for item in sublist])
    demography["Age"] = data
    age_df = demography.groupby("Age").sum()
    return age_df

def populationAffectedByDemography(country,days,model):
    x_data,y_data,df = prediction(country,"Confirmed",days,model,fromNow=True)
    data = []
    infected = list(df["People"])[-1]
    ageFatality = fatalityRate(AGE)
    affected = ageAffected(country)
    l = []
    for age,row in affected.iterrows():
        try:
            rate = float(list(ageFatality[ageFatality["AGE"] == age]["DEATH RATE all cases"])[0].split("%")[0])
        except:
            rate = 0
        l.append((age,rate/100*row["M"]*infected/inhabitantsCountry(country)))
    return pd.DataFrame(l,columns=["Age","Deaths"])
populationAffectedByDemography("Spain",2,expModel)
